In [1]:
import pandas as pd
import numpy as np
import os
import glob
import random

import warnings
warnings.filterwarnings("ignore")

In [2]:
def preprocess_data(data, is_train=True):
    
    temp = data.copy()
    temp = temp[['Hour', 'TARGET', 'DHI', 'DNI', 'WS', 'RH', 'T']]

    if is_train==True:          
    
        temp['Target1'] = temp['TARGET'].shift(-48).fillna(method='ffill')
        temp['Target2'] = temp['TARGET'].shift(-48*2).fillna(method='ffill')
        temp = temp.dropna()
        
        return temp.iloc[:-96]

    elif is_train==False:
        
        temp = temp[['Hour', 'TARGET', 'DHI', 'DNI', 'WS', 'RH', 'T']]
                              
        return temp.iloc[-48:, :]

In [3]:
train = pd.read_csv('./data/train/train.csv')

test = []

for i in range(81):
    file_path = './data/test/' + str(i) + '.csv'
    temp = pd.read_csv(file_path)
    temp = preprocess_data(temp, is_train=False)
    test.append(temp)

df_test = pd.concat(test)

submission = pd.read_csv('./data/sample_submission.csv')

In [4]:
df_train = preprocess_data(train)

df_train.shape, df_test.shape

((52464, 9), (3888, 7))

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

df_train[['Hour','TARGET','DHI','DNI','WS','RH','T']] = scaler.fit_transform(df_train[['Hour','TARGET','DHI','DNI','WS','RH','T']])
df_test[['Hour','TARGET','DHI','DNI','WS','RH','T']] = scaler.fit_transform(df_test[['Hour','TARGET','DHI','DNI','WS','RH','T']])

In [17]:
df_train.describe()

,Hour,TARGET,DHI,DNI,WS,RH,T,Target1,Target2
count,5.246400e+04,5.246400e+04,5.246400e+04,5.246400e+04,5.246400e+04,5.246400e+04,5.246400e+04,52464.000000,52464.000000
mean,-8.464646e-21,1.079082e-14,-8.113808e-15,-5.101947e-15,3.250340e-16,-6.553459e-16,1.129441e-14,17.809249,17.807131
std,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,25.774762,25.773169
min,-1.661325e+00,-6.908433e-01,-6.195877e-01,-6.713882e-01,-1.720252e+00,-2.230011e+00,-2.780037e+00,0.000000,0.000000
25%,-8.306624e-01,-6.908433e-01,-6.195877e-01,-6.713882e-01,-7.394911e-01,-7.762383e-01,-8.151968e-01,0.000000,0.000000
50%,0.000000e+00,-6.908433e-01,-6.195877e-01,-6.713882e-01,-1.790563e-01,3.647369e-02,-2.926063e-02,0.000000,0.000000
75%,8.306624e-01,5.544975e-01,2.172538e-01,6.698280e-01,5.214873e-01,7.244407e-01,7.566755e-01,32.182481,32.180978
max,1.661325e+00,3.185860e+00,4.459175e+00,2.357072e+00,6.686271e+00,1.958068e+00,2.525032e+00,99.913939,99.913939


In [9]:
Day  = df_train.iloc[:, :-2]
Day7 = df_train.iloc[:, -2]
Day8 = df_train.iloc[:, -1]

In [10]:
import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow.keras import layers
from tensorflow.keras.backend import mean, maximum

In [11]:
def quantile_loss(q, y, pred):
    err = (y-pred)
    return mean(maximum(q*err, (q-1)*err), axis=-1)

In [12]:
q_lst = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [13]:
model7 = tf.keras.Sequential([
    layers.Dense(52, activation='relu', input_shape=(Day.shape)),
    layers.Dense(39, activation='relu'),
    layers.Dense(26, activation='relu'),
    layers.Dense(1)
])

In [14]:
results7 = pd.DataFrame()

for q in q_lst:
    model7.compile(loss=lambda y,pred: quantile_loss(q,y,pred), optimizer='adam')
    model7.fit(Day, Day7, epochs=25, batch_size=48, validation_split=0.25)
    pred = pd.DataFrame(model7.predict(df_test))
    results7 = pd.concat([results7, pred], axis=1)

results7.shape

Epoch 1/25
820/820 [==============================] - 2s 2ms/step - loss: 1.5120 - val_loss: 1.5875
Epoch 2/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3823 - val_loss: 1.5717
Epoch 3/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3742 - val_loss: 1.5505
Epoch 4/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3475 - val_loss: 1.5643
Epoch 5/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3401 - val_loss: 1.5648
Epoch 6/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3245 - val_loss: 1.5629
Epoch 7/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3386 - val_loss: 1.5863
Epoch 8/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3355 - val_loss: 1.5726
Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3305 - val_loss: 1.5554
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3150 - val_loss: 1.5585

820/820 [==============================] - 1s 1ms/step - loss: 2.4286 - val_loss: 3.0122
Epoch 2/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4278 - val_loss: 3.0189
Epoch 3/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4144 - val_loss: 3.0156
Epoch 4/25
820/820 [==============================] - 1s 1ms/step - loss: 2.3852 - val_loss: 3.0109
Epoch 5/25
820/820 [==============================] - 1s 1ms/step - loss: 2.3785 - val_loss: 3.0083
Epoch 6/25
820/820 [==============================] - 1s 1ms/step - loss: 2.3805 - val_loss: 2.9827
Epoch 7/25
820/820 [==============================] - 1s 1ms/step - loss: 2.3854 - val_loss: 3.0502
Epoch 8/25
820/820 [==============================] - 1s 1ms/step - loss: 2.3685 - val_loss: 2.9791
Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 2.3564 - val_loss: 3.0099
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 2.3813 - val_loss: 3.0155
Epoch 11/2

820/820 [==============================] - 1s 1ms/step - loss: 2.3496 - val_loss: 3.0457
Epoch 4/25
820/820 [==============================] - 1s 1ms/step - loss: 2.3773 - val_loss: 3.0501
Epoch 5/25
820/820 [==============================] - 1s 1ms/step - loss: 2.3584 - val_loss: 3.0727
Epoch 6/25
820/820 [==============================] - 1s 1ms/step - loss: 2.3555 - val_loss: 3.0989
Epoch 7/25
820/820 [==============================] - 1s 1ms/step - loss: 2.3439 - val_loss: 3.0864
Epoch 8/25
820/820 [==============================] - 1s 1ms/step - loss: 2.3873 - val_loss: 3.0813
Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 2.3575 - val_loss: 3.0605
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 2.3545 - val_loss: 3.1277
Epoch 11/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4047 - val_loss: 3.0881
Epoch 12/25
820/820 [==============================] - 1s 1ms/step - loss: 2.2904 - val_loss: 3.1102
Epoch 13

Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7612 - val_loss: 2.3014
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7535 - val_loss: 2.3945
Epoch 11/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7477 - val_loss: 2.3111
Epoch 12/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7580 - val_loss: 2.3274
Epoch 13/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7448 - val_loss: 2.2930
Epoch 14/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7269 - val_loss: 2.2981
Epoch 15/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7169 - val_loss: 2.3106
Epoch 16/25
820/820 [==============================] - 1s 1ms/step - loss: 1.6846 - val_loss: 2.3424
Epoch 17/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7016 - val_loss: 2.2932
Epoch 18/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7254 - val_loss

820/820 [==============================] - 1s 1ms/step - loss: 0.7212 - val_loss: 0.9554
Epoch 15/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7194 - val_loss: 0.9863
Epoch 16/25
820/820 [==============================] - 1s 1ms/step - loss: 0.6991 - val_loss: 1.0096
Epoch 17/25
820/820 [==============================] - 1s 1ms/step - loss: 0.6961 - val_loss: 0.9501
Epoch 18/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7046 - val_loss: 0.9799
Epoch 19/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7007 - val_loss: 0.9943
Epoch 20/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7105 - val_loss: 0.9879
Epoch 21/25
820/820 [==============================] - 1s 1ms/step - loss: 0.6895 - val_loss: 1.0343
Epoch 22/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7075 - val_loss: 0.9479
Epoch 23/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7029 - val_loss: 0.9403
Ep

(3888, 9)

In [15]:
model8 = tf.keras.Sequential([
    layers.Dense(52, activation='relu', input_shape=(Day.shape)),
    layers.Dense(39, activation='relu'),
    layers.Dense(26, activation='relu'),
    layers.Dense(1)
])

In [16]:
results8 = pd.DataFrame()

for q in q_lst:
    model8.compile(loss=lambda y,pred: quantile_loss(q,y,pred), optimizer='adam')
    model8.fit(Day, Day8, epochs=25, batch_size=48, validation_split=0.25)
    pred = pd.DataFrame(model8.predict(df_test))
    results8 = pd.concat([results8, pred], axis=1)

results8.shape

Epoch 1/25
820/820 [==============================] - 1s 1ms/step - loss: 1.5402 - val_loss: 1.6054
Epoch 2/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4123 - val_loss: 1.5742
Epoch 3/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3869 - val_loss: 1.5528
Epoch 4/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3760 - val_loss: 1.5506
Epoch 5/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3567 - val_loss: 1.5683
Epoch 6/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3746 - val_loss: 1.5868
Epoch 7/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3906 - val_loss: 1.5563
Epoch 8/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3502 - val_loss: 1.5588
Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3749 - val_loss: 1.5503
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3478 - val_loss: 1.5634

820/820 [==============================] - 1s 1ms/step - loss: 2.5303 - val_loss: 3.1053
Epoch 2/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5588 - val_loss: 3.1498
Epoch 3/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5022 - val_loss: 3.1514
Epoch 4/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4997 - val_loss: 3.1271
Epoch 5/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5232 - val_loss: 3.1469
Epoch 6/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5105 - val_loss: 3.1744
Epoch 7/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5064 - val_loss: 3.2199
Epoch 8/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5131 - val_loss: 3.1640
Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4730 - val_loss: 3.1019
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4881 - val_loss: 3.2446
Epoch 11/2

820/820 [==============================] - 1s 1ms/step - loss: 2.4769 - val_loss: 3.2778
Epoch 3/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4931 - val_loss: 3.2927
Epoch 4/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5074 - val_loss: 3.3117
Epoch 5/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4584 - val_loss: 3.3569
Epoch 6/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4683 - val_loss: 3.2902
Epoch 7/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5028 - val_loss: 3.3112
Epoch 8/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4483 - val_loss: 3.2466
Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4460 - val_loss: 3.2470
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4880 - val_loss: 3.2655
Epoch 11/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4828 - val_loss: 3.2474
Epoch 12/

820/820 [==============================] - 1s 1ms/step - loss: 1.8159 - val_loss: 2.4566
Epoch 8/25
820/820 [==============================] - 1s 1ms/step - loss: 1.8065 - val_loss: 2.4742
Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7856 - val_loss: 2.4990
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7503 - val_loss: 2.4743
Epoch 11/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7983 - val_loss: 2.4227
Epoch 12/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7609 - val_loss: 2.4932
Epoch 13/25
820/820 [==============================] - 1s 1ms/step - loss: 1.8343 - val_loss: 2.4590
Epoch 14/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7782 - val_loss: 2.5044
Epoch 15/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7986 - val_loss: 2.4701
Epoch 16/25
820/820 [==============================] - 1s 1ms/step - loss: 1.8047 - val_loss: 2.4290
Epoc

820/820 [==============================] - 1s 1ms/step - loss: 0.7316 - val_loss: 1.0992
Epoch 13/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7375 - val_loss: 1.0676
Epoch 14/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7318 - val_loss: 1.0524
Epoch 15/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7283 - val_loss: 1.1376
Epoch 16/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7315 - val_loss: 1.1010
Epoch 17/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7194 - val_loss: 1.1269
Epoch 18/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7181 - val_loss: 1.0703
Epoch 19/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7232 - val_loss: 1.0859
Epoch 20/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7170 - val_loss: 1.1163
Epoch 21/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7237 - val_loss: 1.0260
Ep

(3888, 9)

In [18]:
submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = results7.sort_index().values
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = results8.sort_index().values
submission

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
0,0.csv_Day7_0h00m,-0.002960,-0.016055,-0.000115,-0.018353,-0.002098,-0.000338,0.021391,0.001286,0.000202
1,0.csv_Day7_0h30m,-0.004104,-0.015207,0.000150,-0.018385,-0.001240,-0.000360,0.021963,0.001358,0.000190
2,0.csv_Day7_1h00m,-0.005060,-0.014430,-0.000417,-0.017667,-0.000948,-0.000680,0.019669,0.001067,0.000267
3,0.csv_Day7_1h30m,-0.005836,-0.015146,-0.000783,-0.017920,-0.001732,-0.000453,0.019116,0.000929,0.000282
4,0.csv_Day7_2h00m,-0.006547,-0.013049,-0.000697,-0.017040,-0.002229,-0.000625,0.016915,0.000569,0.000369
...,...,...,...,...,...,...,...,...,...,...
7771,80.csv_Day8_21h30m,-0.001220,-0.001721,-0.007287,-0.003003,0.003594,0.000767,0.009749,0.016869,0.016784
7772,80.csv_Day8_22h00m,-0.001285,-0.002055,-0.007726,-0.003169,0.004163,0.000458,0.010621,0.018762,0.016627
7773,80.csv_Day8_22h30m,-0.001226,-0.002017,-0.007632,-0.002956,0.004063,0.000236,0.010631,0.018443,0.016556
7774,80.csv_Day8_23h00m,-0.001308,-0.002319,-0.007846,-0.002696,0.004897,-0.000411,0.011550,0.020165,0.016957


In [19]:
submission.to_csv('./submission/submission_20210113-3.csv', index=False)